In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Input, Flatten, concatenate,LSTM,Dropout
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
#mae(ts_data[['cleanliness','location','rooms','service','value']], pred_test_data )

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
kf = KFold(n_splits=5,random_state=24,shuffle=True)
from sklearn.neighbors import DistanceMetric
dist = DistanceMetric.get_metric('hamming')
mae_hist = []
mae_cleanliness = []
mae_loc = []
mae_rooms = []
mae_srv = []

mae_value = []

rmse_hist = []
rmse_cleanliness = []
rmse_loc = []
rmse_rooms = []
rmse_srv = []

rmse_value = []
k = 3
f = 1

for i in range(1,6):
    train_data = pd.read_csv('/content/drive/MyDrive/Trip_fold_with_sentiment'+str(i)+'_train.csv')
    y_train  =train_data[['cleanliness','location','rooms','service','value']].values
    test_data = pd.read_csv('/content/drive/MyDrive/Trip_fold_with_sentiment'+str(i)+'_test.csv')
    y_test = test_data[['cleanliness','location','rooms','service','value']].values
    pred_Bert_cf = []
    print('fold',i)
    cmp=0
    for idx,row in test_data.iterrows():
      cmp+=1
      precedent_data = train_data[train_data['user_id']==row['user_id']]
      precedent_hotels = precedent_data.offering_id.values
      prcd_reviews = precedent_data.text.values
      hotel_id = row['offering_id']
      
      if hotel_id in train_data.offering_id.values:
        precedent_data = train_data[train_data['user_id']==row['user_id']]

        if precedent_data.shape[0] > 0:

          precedent_hotels = precedent_data.offering_id.values
          prcd_reviews = precedent_data.sentiment_score.values
          
        
          prcedent_data_sentiment_review = {}
          for h_id,rev in zip(precedent_hotels,prcd_reviews):
            
            
            prcedent_data_sentiment_review[h_id] = rev
            
          
          user = row['user_id']

          maybe_sim_users = []
          sim = {}
          if cmp%100==0:
            print('rest',test_data.shape[0] - cmp)

          users = train_data[train_data['offering_id']==hotel_id].user_id.unique()
          for user1 in users:
            if (user1 != user):
              

              user1_data = train_data[train_data['user_id']==user1]
              sim_items = set(precedent_hotels).intersection(set(user1_data.offering_id.values))
              if len(sim_items) > 1:
                  
                  maybe_sim_users.append(user1)
                
                  user_rat = []
                  user1_rat = []
                  for item in sim_items:
                    
                    user_rat.append(prcedent_data_sentiment_review[item])

                  for item in sim_items:
                    rev = user1_data[user1_data['offering_id']==item].sentiment_score.values
                    
                    
                    user1_rat.append(rev[0])
                  
                  if (1 - dist.pairwise([user_rat],[user1_rat])[0][0])> 0.5:
                    sim[user1] = 1 - dist.pairwise([user_rat],[user1_rat])[0][0]
          
          if len(sim.keys())>0: #similar users
            rat = [0.0,0.0,0.0,0.0,0.0]
            sum_sim = 0.0
            for user1 in list(sim.keys())[:k]:
              rat += sim[user1] * train_data[(train_data['user_id']==user1) & (train_data['offering_id']==hotel_id ) ][['cleanliness','location','rooms','service','value']].values[0]
              sum_sim+= sim[user1]
            rat = rat / sum_sim
          
          
          else : #no similar users
            rat = [0.0,0.0,0.0,0.0,0.0]
            for i,col in zip([0,1,2,3,4],['cleanliness','location','rooms','service','value']):
              rat[i] = np.mean(train_data[train_data['offering_id']==hotel_id][[col]].values)
          
        else: #new_user
            rat = [0.0,0.0,0.0,0.0,0.0]
            for i,col in zip([0,1,2,3,4],['cleanliness','location','rooms','service','value']):
              rat[i] = np.mean(train_data[train_data['offering_id']==hotel_id][[col]].values)  

      elif train_data[train_data['user_id']==row['user_id']].shape[0]>0: #new_item
        rat = [0.0,0.0,0.0,0.0,0.0]
        for i,col in zip([0,1,2,3,4],['cleanliness','location','rooms','service','value']):
            rat[i] = np.mean(precedent_data[[col]].values)
      else: #new_user , new_item
        #rnd_petubation = random.uniform(-1,1)
        #val = rnd_petubation + 3.0
        rat = [train_data['cleanliness'].mean(),train_data['location'].mean(),train_data['rooms'].mean()
            ,train_data['service'],train_data['value']]
      pred_Bert_cf.append(rat.copy())

    pred_Bert_cf = np.array(pred_Bert_cf)
    pred_test_data = np.copy(pred_Bert_cf)
    mae_cleanliness.append(mae(pred_test_data[:,0],y_test[:,0]))
    mae_loc.append(mae(pred_test_data[:,1],y_test[:,1]))
    mae_rooms.append(mae(pred_test_data[:,2],y_test[:,2]))
    mae_srv.append(mae(pred_test_data[:,3],y_test[:,3]))
    
    mae_value.append(mae(pred_test_data[:,4],y_test[:,4]))
    mae_hist.append(mae(pred_test_data,y_test))
      
    rmse_cleanliness.append(mse(pred_test_data[:,0],y_test[:,0], squared=False))
    rmse_loc.append(mse(pred_test_data[:,1],y_test[:,1], squared=False))
    rmse_rooms.append(mse(pred_test_data[:,2],y_test[:,2], squared=False))
    rmse_srv.append(mse(pred_test_data[:,3],y_test[:,3], squared=False))
  
    rmse_value.append(mse(pred_test_data[:,4],y_test[:,4], squared=False))
    rmse_hist.append(mse(pred_test_data,y_test, squared=False))

    print(mae_hist)
    print(rmse_hist)
      




    


/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_distance_metric.py:14: FutureWarning: sklearn.neighbors.DistanceMetric has been moved to sklearn.metrics.DistanceMetric in 1.0. This import path will be removed in 1.3
  category=FutureWarning,


fold 1
rest 1834
rest 1734
rest 1634
rest 1534
rest 1434
rest 1334
rest 1234
rest 1134
rest 1034
rest 934
rest 734
rest 634
rest 534
rest 434
rest 334
rest 234
rest 134
rest 34
[0.8279933581120865]
[1.1292571438017642]
fold 2
rest 1834
rest 1734
rest 1634
rest 1534
rest 1334
rest 1234
rest 1134
rest 1034
rest 934
rest 834
rest 734
rest 634
rest 534
rest 434
rest 334
rest 234
rest 134
rest 34
[0.8279933581120865, 0.8141036991423471]
[1.1292571438017642, 1.1104464243017358]
fold 3
rest 1834
rest 1734
rest 1634
rest 1534
rest 1434
rest 1334
rest 1234
rest 1134
rest 1034
rest 834
rest 734
rest 634
rest 534
rest 434
rest 334
rest 234
rest 134
rest 34
[0.8279933581120865, 0.8141036991423471, 0.8280769645744359]
[1.1292571438017642, 1.1104464243017358, 1.1288463747214934]
fold 4
rest 1833
rest 1733
rest 1633
rest 1533
rest 1433
rest 1333
rest 1233
rest 1133
rest 1033
rest 933
rest 833
rest 733
rest 633
rest 533
rest 433
rest 333
rest 233
rest 133
rest 33
[0.8279933581120865, 0.814103699142347

In [ ]:
print('mean_mae',mae_hist , np.mean(mae_hist))
print('mae_cleanliness',mae_cleanliness ,np.mean(mae_cleanliness ))
print('mae_loc',mae_loc,np.mean(mae_loc))
print('mae_rooms',mae_rooms,np.mean(mae_rooms))
print('mae_srv',mae_srv,np.mean(mae_srv))

print('mae_value',mae_value,np.mean(mae_value))

print('rmse_hist',rmse_hist,np.mean(rmse_hist))
print('rmse_cleanliness',rmse_cleanliness,np.mean(rmse_cleanliness))
print('rmse_loc',rmse_loc,np.mean(rmse_loc))
print('rmse_rooms',rmse_rooms,np.mean(rmse_rooms))
print('rmse_srv',rmse_srv,np.mean(rmse_srv))

print('rmse_value',rmse_value,np.mean(rmse_value))

mean_mae [0.8279933581120865, 0.8141036991423471, 0.8280769645744359, 0.8186444212659918, 0.8341973154394932] 0.8246031517068708
mae_cleanliness [0.7665987254933108, 0.7615676777412173, 0.7526866445132392, 0.7365002466184655, 0.7837204682615739] 0.7602147525255614
mae_loc [0.6442101388177961, 0.6424114256733091, 0.625513414908339, 0.6463652083482269, 0.6319033495125962] 0.6380807074520535
mae_rooms [0.8689010656187979, 0.8606580564362766, 0.8872985003016118, 0.8706293546168894, 0.8624858569371134] 0.8699945667821378
mae_srv [0.8832320145569756, 0.8662905351823716, 0.9002263657726345, 0.8773861515418138, 0.9089110590112717] 0.8872092252130134
mae_value [0.9770248460735528, 0.93959080067856, 0.9746598973763595, 0.9623411452045626, 0.9839658434749139] 0.9675165065615896
rmse_hist [1.1292571438017642, 1.1104464243017358, 1.1288463747214934, 1.1065956690507595, 1.1290257450674697] 1.1208342713886443
rmse_cleanliness [1.0713165833919485, 1.0739141036645958, 1.0563488911608252, 1.035709428029